<a href="https://colab.research.google.com/github/giuliovv/ANNDL_competition_1/blob/master/toma_models/custom_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

colab = "True" #@param ['True','False']
if colab == "True":
  from google.colab import drive
  drive.mount('/gdrive')
  %cd /gdrive/MyDrive/Colab Notebooks
  if not os.path.isdir('training'):
    !unzip dataset.zip

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/Colab Notebooks


In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
dataset_dir = 'training'

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from sklearn.model_selection import train_test_split
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']
y = tf.keras.utils.to_categorical(range(len(labels)))
y

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],
      dtype=float32)

In [ ]:
import shutil

get_plants = False

if not "test" in os.listdir():
  print("No test directory!")
  for label in labels:
    print(label)
    image_names = [pic for pic in os.listdir("training/"+label)]
    train_images, test_images = train_test_split(image_names, test_size=0.05)
    if not 'test/'+label in os.listdir():
      try:
        os.makedirs(os.path.abspath(os.getcwd())+'/test/'+label+'/')
      except OSError:
        print("Failed to create a dir for ", '/test/'+label+'/')
      else:
        print("Success creating folder ", label)
    for name in test_images:
      shutil.move(os.path.abspath(os.getcwd())+'/training/'+label+'/'+name, os.path.abspath(os.getcwd())+'/test/'+label+'/'+name)
  print("Transfered all testing data!")

if get_plants:
  ! pip install -q tfds-nightly
  import tensorflow_datasets as tfds
  from PIL import Image
  ds = tfds.load('plant_village', split='train')
  builder = tfds.builder('plant_village')
  info = builder.info
  labels_tf = info.features["label"].names
  if not 'test_expanded/' in os.listdir():
      try:
        os.makedirs(os.path.abspath(os.getcwd())+'/test_expanded/')
      except OSError:
        print("Failed to create a dir for /test_expanded/")
      else:
        print("Success creating folder test_expanded")
  image_number = 54303
  ds = ds.take(54303)
  for el in ds:
    label_new_ds = labels_tf[el["label"].numpy()]
    for just_fruit_name in labels:
      if just_fruit_name in label_new_ds:
        if not just_fruit_name in os.listdir("test_expanded"):
          try:
            os.makedirs(os.path.abspath(os.getcwd())+'/test_expanded/'+just_fruit_name+'/')
          except OSError:
            print("Failed to create a dir for ", '/test_expanded/'+just_fruit_name+'/')
          else:
            print("Success creating folder ", just_fruit_name)
        im = Image.fromarray(el["image"].numpy())
        im.save(os.path.abspath(os.getcwd())+'/test_expanded/'+just_fruit_name+"/"+str(image_number)+".jpeg")
        break
    if image_number % 1000 == 0:
      print(image_number)
    image_number += 1


In [ ]:
BATCH_SIZE = 64
VALIDATE_BATCH_SIZE = BATCH_SIZE

train_generator = ImageDataGenerator(
## when rotating some pixels remain unfilled, to fill them the default function is to take the "nearest pixel color", this maybe screws up the learning process so I left it for later training
                                    #featurewise_center=True,
                                    #featurewise_std_normalization=True,
                                    rotation_range=90,
                                    fill_mode='nearest',
                                    brightness_range=(0.2,1.8),
                                    channel_shift_range=150,
                                    shear_range=0.7,
                                    zoom_range=0.5,
                                    width_shift_range=0.5, 
                                    height_shift_range=0.5,
                                    horizontal_flip=True, 
                                    vertical_flip=True,
                                    validation_split=0.1,
                                    preprocessing_function=tf.keras.applications.xception.preprocess_input) # VGG16 preprocessing
test_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.xception.preprocess_input)


#train_generator.fit(X, seed=42)

traingen = train_generator.flow_from_directory('training',
                                              target_size=(256, 256),
                                              class_mode='categorical',
                                              classes=labels,
                                              subset='training',
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              seed=42)

validgen = train_generator.flow_from_directory('training',
                                              target_size=(256, 256),
                                              class_mode='categorical',
                                              classes=labels,
                                              subset='validation',
                                              batch_size=VALIDATE_BATCH_SIZE,
                                              shuffle=True,
                                              seed=42)

testgen = test_generator.flow_from_directory('test',
                                            target_size=(256, 256),
                                            class_mode='categorical',
                                            classes=labels,
                                            batch_size=1,
                                            shuffle=False,
                                            seed=42)

Found 15158 images belonging to 14 classes.
Found 1677 images belonging to 14 classes.
Found 893 images belonging to 14 classes.


In [ ]:
n_steps = traingen.samples / BATCH_SIZE
n_val_steps = validgen.samples / VALIDATE_BATCH_SIZE

In [ ]:
n_classes = len(labels)

In [ ]:
load = False

if load:
  model = tf.keras.models.load_model("custom_net_backup")

In [ ]:
import keras
from tensorflow.keras import layers

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(512, activation="relu")(x)

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(inputs, outputs)


model = make_model(input_shape=(256,256,3), num_classes=n_classes)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 rescaling_2 (Rescaling)        (None, 64, 64, 3)    0           ['input_3[0][0]']                
                                                                                                  
 conv2d_12 (Conv2D)             (None, 32, 32, 32)   896         ['rescaling_2[0][0]']            
                                                                                                  
 batch_normalization_22 (BatchN  (None, 32, 32, 32)  128         ['conv2d_12[0][0]']              
 ormalization)                                                                              

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks = []

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)
    callbacks.append(es_callback)
    
#LRPlateau
lr_plateau_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=3,
    min_lr=0,
)  

callbacks.append(lr_plateau_callback)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)

callbacks.append(tensorboard_callback)

backup = tf.keras.callbacks.experimental.BackupAndRestore(
    "custom_net_backup"
)

In [ ]:
model.fit(traingen, epochs=6, steps_per_epoch=n_steps, batch_size=BATCH_SIZE, validation_data=validgen, callbacks=callbacks+[backup])

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/6


In [ ]:
if not load:
  model.save("custom_net_backup")

In [ ]:
model.evaluate(testgen)